In [ ]:
# 특정 종목의 코드를 받으면, 해당 코드의 관련 정보를 확인 후 매수 추천여부를 알려준다.

# 1. 기능
# = 코드 입력 시 해당 주가 매수추천여부 출력 (financeDatareader 이용, 시세차익) / 배당 per 등 배당 관련 정보 출력 / 전체 주가그래프 제시


In [1]:
# 필요한 라이브러리

import FinanceDataReader as web
from datetime import date, timedelta
import matplotlib.pyplot as plt
import datetime as dt
import pandas as pd
import time

import csv
import urlib
from bs4 import BeautifulSoup as bs

%matplotlib inline

In [7]:
# 필요값 (전역변수)

market =""
today = date.today()
startday = date.today() - timedelta(1)
yesterday = date.today() - timedelta(1)

In [15]:
# 전체 가동 코드

print("want to decide your settings? it will create data automatically.(y/n)")

while(True):
    menubtn = input()
    print("")
    if menubtn == "y":
        settings()
        createData(market)
        break
    elif menubtn == "n":
        break
    else:
        print("selected wrong button, want to decide your settings?(y/n)")
        
print("select ur menus.")
print("")
print("seeAll = show all stock datas of market which u selected.")
print("search = search some stocks which you want, it will show charts and recent dividend infos.")

while(True):
    menubtn = input()
    if menubtn == "seeAll":
        print("s")
    elif menubtn == "search":
        print("a")
    else:
        print("you just selected wrong menu.")
        print("")

want to decide your settings? it will create data automatically.(y/n)


 y



market list : 
KRX(한국전체), KOSPI(코스피전체), KOSDAQ(코스닥전체), KONEX(코넥스전체)
NASDAQ(나스닥종목), NYSE(뉴욕증권거래소), AMEX(AMEX종목), SP500(S&P500 종목)

'dayago' decieds your startday to crawl data. we recommand to set 'dayago' at least 1 year. (365)

set your target market.


 NASDAQ


set your 'dayago'.


...

  2%|█▏                                                                             | 60/3878 [00:00<00:09, 419.23it/s]


**** Data has been set. make sure ur market. if it incorrect with the list above, then it will occur some errors. ****

market :  NASDAQ
startday(dayago) :  2020-06-26 ( 360 )

loading entire stockcode data from market..
list none-exists. load marketList which you requested.


100%|█████████████████████████████████████████████████████████████████████████████| 3878/3878 [00:07<00:00, 488.39it/s]


loading complete. making lists..
list created, initialize to next steps.
select ur menus.

seeAll = show all stock datas of market which u selected.
search = search some stocks which you want, it will show charts and dividend infos.


KeyboardInterrupt: Interrupted by user

In [ ]:
# seeAll - GC 접근 여부 확인해 줄 로직 설정

In [ ]:
# search - 배당정보 형성 클래스 생성
# https://m.blog.naver.com/silvury/221312883764

def getDividendData(stockCode):
    data = []
    url = "https://finance.yahoo.com/quote/" + stockcode + "?p=" + stockcode
    rows = bs(urllib.urlopen(url).read()).findAll('table')[1].tbody.findAll('tr')

In [16]:
# 가동일 기준 데이터를 로드, 스톡리스트를 생성해줄 클래스 생성

# market = url 통해 input값 받아옴
# dayago = 위와 동일

def createData(market):
    
    codefilename = './stockDataSet/marketList_' + str(market) + '.csv'
    
    # 목표 시장 선택 및 데이터 로드
    print("loading entire stockcode data from market..")
    try:
        with open(codefilename, 'r', newline='') as file: # file을 여는게 아니라 있는지 확인하는 기능으로 치환
            print("list already exists, proceed to next steps.")
    except:
        try:
            print("list none-exists. load marketList which you requested.")
            targetMarket = web.StockListing(market)
            print("loading complete. making lists..")

            targetMarket.to_csv('./stockDataSet/marketList_' + str(market) + '.csv')
            print("list created, initialize to next steps.")

        except:
            time.sleep(3)
            print("loading error occured, please try again or check ur status.")

In [8]:
# 기본값 입력받을 클래스
def settings():
    print("market list : ")
    print("KRX(한국전체), KOSPI(코스피전체), KOSDAQ(코스닥전체), KONEX(코넥스전체) - 배당 구현중")
    print("NASDAQ(나스닥종목), NYSE(뉴욕증권거래소), AMEX(AMEX종목), SP500(S&P500 종목)")
    print("")
    print("'dayago' decieds your startday to crawl data. we recommand to set 'dayago' at least 1 year. (365)")
    print("")

    
    print("set your target market.")
    global market
    market = input()
    
    print("set your 'dayago'.")
    dayago = input()

    global startday
    startday = date.today() - timedelta(int(dayago))
        
    print("")
    print("**** Data has been set. make sure ur market. if it doesn't match with the list above, then it will occur some errors. ****")
    print("")

    print("market : ", market)
    print("startday(dayago) : ", startday, "(", dayago, ")")
    print("")